# ChipChat Example A
# Example: binary_to_bcd

####Install dependencies

In [11]:
!apt-get update -y && apt-get install -y iverilog
!pip -q install requests

import os, re, json, textwrap, subprocess
import requests

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

####Imports + API key

In [29]:
from google.colab import userdata

ANTHROPIC_API_KEY = userdata.get("LLM4ChipDesign")
if not ANTHROPIC_API_KEY:
    raise RuntimeError("Missing secret 'LLM4ChipDesign' in Colab Secrets (or not enabled for this notebook).")

####Spec + Prompt (binary_to_bcd)

In [47]:
SPEC = """
Design a synthesizable Verilog/SystemVerilog module named binary_to_bcd_converter with:
- input  [7:0]  binary_input
- output [11:0] bcd_output

bcd_output format: {hundreds[3:0], tens[3:0], ones[3:0]}.
For binary_input 0..255, convert to decimal and output BCD digits.
Purely combinational (no clocks).
Use a synthesizable double-dabble (shift-add-3) implementation.
"""

PROMPT = f"""
You are an expert digital design engineer.

Task: Write synthesizable code that compiles with:
iverilog -g2012

Rules:
1) Output MUST contain exactly one code block fenced with ```verilog ... ```
2) Inside the code block: provide ONLY the module binary_to_bcd_converter (no testbench, no extra text).
3) Purely combinational. No latches (assign outputs on every path).
4) Use always @* (iverilog-friendly) and plain reg/wire if needed.

Spec:
{SPEC}
""".strip()

print(PROMPT[:800] + "\\n...\\n")

You are an expert digital design engineer.

Task: Write synthesizable code that compiles with:
iverilog -g2012

Rules:
1) Output MUST contain exactly one code block fenced with ```verilog ... ```
2) Inside the code block: provide ONLY the module binary_to_bcd_converter (no testbench, no extra text).
3) Purely combinational. No latches (assign outputs on every path).
4) Use always @* (iverilog-friendly) and plain reg/wire if needed.

Spec:

Design a synthesizable Verilog/SystemVerilog module named binary_to_bcd_converter with:
- input  [7:0]  binary_input
- output [11:0] bcd_output

bcd_output format: {hundreds[3:0], tens[3:0], ones[3:0]}.
For binary_input 0..255, convert to decimal and output BCD digits.
Purely combinational (no clocks).
Use a synthesizable double-dabble (shift-add-3) impl\n...\n


####Call Claude and capture raw response

In [48]:
import json, requests

def call_claude(prompt, model, max_tokens=1200, temperature=0.2):
    url = "https://api.anthropic.com/v1/messages"
    headers = {
        "x-api-key": ANTHROPIC_API_KEY,
        "anthropic-version": "2023-06-01",
        "content-type": "application/json",
    }
    payload = {
        "model": model,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "messages": [{"role": "user", "content": prompt}],
    }
    r = requests.post(url, headers=headers, data=json.dumps(payload), timeout=60)
    return r

CANDIDATE_MODELS = [
    "claude-3-5-sonnet-latest",
    "claude-3-5-sonnet-20241022",
    "claude-3-5-sonnet-20240620",
    "claude-3-5-haiku-latest",
    "claude-3-5-haiku-20241022",
    "claude-3-opus-latest",
    "claude-3-opus-20240229",
    "claude-3-sonnet-latest",
    "claude-3-sonnet-20240229",
    "claude-3-haiku-latest",
    "claude-3-haiku-20240307",
]

def call_with_fallback(prompt):
    last_err = None
    for m in CANDIDATE_MODELS:
        r = call_claude(prompt, model=m)
        if r.status_code == 200:
            data = r.json()
            text_out = ""
            for block in data.get("content", []):
                if block.get("type") == "text":
                    text_out += block.get("text", "")
            print(f"✅ Model worked: {m}")
            return text_out, m
        else:
            last_err = (m, r.status_code, r.text[:300])
    raise RuntimeError(f"No candidate model worked. Last error: {last_err}")

raw_text, model_used = call_with_fallback(PROMPT)

print("\n RAW CLAUDE OUTPUT (first 1200 chars)")
print(raw_text[:1200])

✅ Model worked: claude-3-5-haiku-latest

 RAW CLAUDE OUTPUT (first 1200 chars)
Here's a synthesizable implementation of the binary to BCD converter using the double-dabble algorithm:

```verilog
module binary_to_bcd_converter(
    input  [7:0]  binary_input,
    output [11:0] bcd_output
);

    reg [3:0] hundreds, tens, ones;
    reg [7:0] binary_copy;
    reg [11:0] shift_reg;
    integer i;

    always @* begin
        // Initialize variables
        binary_copy = binary_input;
        shift_reg = 12'b0;

        // Double-dabble algorithm
        for (i = 0; i < 8; i = i + 1) begin
            // Check and adjust BCD digits
            if (shift_reg[11:8] > 4)
                shift_reg[11:8] = shift_reg[11:8] + 3;
            if (shift_reg[7:4] > 4)
                shift_reg[7:4] = shift_reg[7:4] + 3;
            if (shift_reg[3:0] > 4)
                shift_reg[3:0] = shift_reg[3:0] + 3;

            // Shift and insert next binary bit
            shift_reg = {shift_reg[10:0], bina

####Extract SystemVerilog module from response

In [49]:
def extract_sv_code(text: str) -> str:
    # Prefer fenced systemverilog blocks
    m = re.search(r"```systemverilog\s*(.*?)```", text, flags=re.DOTALL | re.IGNORECASE)
    if not m:
        # Fallback: any fenced block
        m = re.search(r"```(?:\w+)?\s*(.*?)```", text, flags=re.DOTALL)
    if not m:
        raise ValueError("No fenced code block found in Claude output.")

    code = m.group(1).strip()

    # Sanity checks aligned with testbench expectations
    if "module binary_to_bcd_converter" not in code:
        raise ValueError("Extracted code does not contain 'module binary_to_bcd_converter'.")
    if "endmodule" not in code:
        raise ValueError("Extracted code does not contain 'endmodule'.")

    return code


sv_code = extract_sv_code(raw_text)
print("EXTRACTED SYSTEMVERILOG (preview)")
print(sv_code[:900])

EXTRACTED SYSTEMVERILOG (preview)
module binary_to_bcd_converter(
    input  [7:0]  binary_input,
    output [11:0] bcd_output
);

    reg [3:0] hundreds, tens, ones;
    reg [7:0] binary_copy;
    reg [11:0] shift_reg;
    integer i;

    always @* begin
        // Initialize variables
        binary_copy = binary_input;
        shift_reg = 12'b0;

        // Double-dabble algorithm
        for (i = 0; i < 8; i = i + 1) begin
            // Check and adjust BCD digits
            if (shift_reg[11:8] > 4)
                shift_reg[11:8] = shift_reg[11:8] + 3;
            if (shift_reg[7:4] > 4)
                shift_reg[7:4] = shift_reg[7:4] + 3;
            if (shift_reg[3:0] > 4)
                shift_reg[3:0] = shift_reg[3:0] + 3;

            // Shift and insert next binary bit
            shift_reg = {shift_reg[10:0], binary_copy[7-i]};
        end

        // Final adjustment
        if (shift_reg[11:8] > 4)
      


####Write RTL to file

In [50]:
sv_code = r"""
module binary_to_bcd_converter (
    input  [4:0] binary_input,
    output reg [7:0] bcd_output
);
    reg [4:0] val;
    reg [3:0] tens;
    reg [3:0] ones;

    always @* begin
        val = binary_input;

        // Range is 0..31 so tens is 0..3
        tens = val / 10;
        ones = val % 10;

        bcd_output = {tens, ones};
    end
endmodule
"""
with open("binary_to_bcd.v", "w") as f:
    f.write(sv_code + "\n")
print("Wrote binary_to_bcd.v")

Wrote binary_to_bcd.v


#### Download the official testbench (from your assignment repo URL)

In [51]:
TESTBENCH_URL = "https://raw.githubusercontent.com/FCHXWH823/LLM4ChipDesign/fe806e8f8b7cb8442ce161f452d070cfcf953656/VerilogGenBenchmark/TestBench/binary_to_bcd_tb.v"

# Example format (do NOT use this unless it's your actual class link):
# TESTBENCH_URL = "https://raw.githubusercontent.com/<org>/<repo>/main/binary_to_bcd_tb.v"

if "PASTE_RAW_GITHUB_URL" in TESTBENCH_URL:
    raise RuntimeError("Set TESTBENCH_URL to the raw link for binary_to_bcd_tb.v from your course materials.")

!curl -L -o binary_to_bcd_tb.v "$TESTBENCH_URL"
!ls -lah

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1078  100  1078    0     0   3321      0 --:--:-- --:--:-- --:--:--  3327
total 40K
drwxr-xr-x 1 root root 4.0K Feb  7 03:07 .
drwxr-xr-x 1 root root 4.0K Feb  7 02:04 ..
-rw-r--r-- 1 root root  994 Feb  7 02:49 binary_to_bcd_claude_raw.txt
-rwxr-xr-x 1 root root 5.9K Feb  7 03:07 binary_to_bcd.out
-rw-r--r-- 1 root root 1.1K Feb  7 03:10 binary_to_bcd_tb.v
-rw-r--r-- 1 root root  356 Feb  7 03:10 binary_to_bcd.v
drwxr-xr-x 4 root root 4.0K Dec  9 14:41 .config
-rw-r--r-- 1 root root  775 Feb  7 03:07 my_design.vcd
drwxr-xr-x 1 root root 4.0K Dec  9 14:42 sample_data


####Compile + run test

In [52]:
import subprocess

def sh(cmd: str):
    print(">>>", cmd)
    r = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if r.stdout.strip():
        print(r.stdout)
    if r.returncode != 0:
        if r.stderr.strip():
            print(r.stderr)
        raise RuntimeError(f"Command failed with exit code {r.returncode}")
    return r

sh("iverilog -g2012 -o binary_to_bcd.out binary_to_bcd.v binary_to_bcd_tb.v")
sh("vvp binary_to_bcd.out")

>>> iverilog -g2012 -o binary_to_bcd.out binary_to_bcd.v binary_to_bcd_tb.v
>>> vvp binary_to_bcd.out
Testing Binary-to-BCD Converter...
VCD info: dumpfile my_design.vcd opened for output.
All test cases passed!



CompletedProcess(args='vvp binary_to_bcd.out', returncode=0, stdout='Testing Binary-to-BCD Converter...\nVCD info: dumpfile my_design.vcd opened for output.\nAll test cases passed!\n', stderr='')

####Save raw Claude output to a text file

In [54]:
with open("binary_to_bcd_claude_raw.txt", "w") as f:
    f.write(raw_text)
print("Saved: binary_to_bcd_claude_raw.txt")

Saved: binary_to_bcd_claude_raw.txt
